# GLACIS Online Attestation Demo (Production Mode)

**What is GLACIS?** A library that creates tamper-proof receipts for AI operations.
Prove what your AI did, what data it saw - without sensitive data leaving your environment.

## What You'll Learn
1. **Online Attestation** - Server-witnessed attestations with Merkle proofs
2. **Control Stack** - PII/PHI redaction + jailbreak detection
3. **Third-Party Verification** - Badge URL for external verification
4. **Query Transparency Log** - Search past attestations
5. **Compliance Export** - Export attestations for audit reports

## Core Concept: "Hash Locally, Prove Globally"
- Input/output data is hashed locally (SHA-256)
- Only hashes are transmitted to GLACIS server
- Receipts are included in a Merkle tree
- `witness_status` = `WITNESSED` with Merkle inclusion proofs

**Just getting started?** Try the [Offline Attestation Demo](offline_attestation_demo.ipynb) first - no API key required.

---
## Setup

```bash
pip install glacis[openai,controls]   # OpenAI + PII redaction + jailbreak detection
```

**Required Environment Variables:**
- `GLACIS_API_KEY` - Get yours at https://glacis.io
- `OPENAI_API_KEY` - For OpenAI integration

For full NER-based redaction, also install the spaCy model:
```bash
python -m spacy download en_core_web_md
```

In [1]:
import os
from pathlib import Path

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    env_path = Path("../.env") if Path("../.env").exists() else Path(".env")
    load_dotenv(env_path)
    print(f"Loaded environment from: {env_path}")
except ImportError:
    print("python-dotenv not installed, using system environment variables")

# Check for required API keys
GLACIS_API_KEY = os.environ.get("GLACIS_API_KEY", "")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")

if not GLACIS_API_KEY:
    print("WARNING: No GLACIS_API_KEY found. Get one at https://glacis.io")
else:
    print("GLACIS API key loaded")

if not OPENAI_API_KEY:
    print("WARNING: No OPENAI_API_KEY found.")
else:
    print("OpenAI API key loaded")

Loaded environment from: .env
GLACIS API key loaded
OpenAI API key loaded


---
## 1. Basic Online Attestation

Create a server-witnessed attestation. The receipt includes Merkle tree proofs for third-party verification.

In [2]:
if GLACIS_API_KEY:
    from glacis import Glacis

    # Create an online Glacis client
    glacis = Glacis(api_key=GLACIS_API_KEY)
    print(f"Glacis client mode: {glacis.mode}")
else:
    print("Skipped - no GLACIS_API_KEY")

Glacis client mode: GlacisMode.ONLINE


In [ ]:
if GLACIS_API_KEY:
    # Simulate an AI interaction
    prompt = "What is the capital of France?"
    response = "The capital of France is Paris."

    # Create a witnessed attestation
    receipt = glacis.attest(
        service_id="demo-app",
        operation_type="inference",
        input={"prompt": prompt},
        output={"response": response},
        metadata={"model": "gpt-4", "temperature": 0.7},
    )

    print("Attestation created!")
    print(f"  Receipt ID:       {receipt.id}")
    print(f"  Evidence Hash:    {receipt.evidence_hash[:32]}...")
    print(f"  Leaf Index:       {receipt.leaf_index}")
    print(f"  Tree Size:        {receipt.tree_size}")

    # Show Merkle proof details
    if receipt.receipt and receipt.receipt.transparency_proofs:
        proofs = receipt.receipt.transparency_proofs
        print(f"\n  Merkle Proof:")
        print(f"    Root Hash: {proofs.sth_curr.root_hash[:32]}...")
        print(f"    Tree Size: {proofs.sth_curr.tree_size}")
else:
    print("Skipped - no GLACIS_API_KEY")

Attestation created!


AttributeError: 'AttestReceipt' object has no attribute 'evidence_hash'

In [ ]:
if GLACIS_API_KEY:
    from glacis import GlacisApiError

    try:
        # Verify the receipt (includes Merkle proof verification)
        result = glacis.verify(receipt)

        print("Verification result:")
        print(f"  Overall Valid:   {result.valid}")
        if result.verification:
            print(f"  Signature Valid: {result.verification.signature_valid}")
            print(f"  Proof Valid:     {result.verification.proof_valid}")
        if result.proof:
            print(f"  Merkle Proof:    included")
    except GlacisApiError as e:
        # This can happen in sandbox/dev mode
        print(f"Server verification not available: {e}")
        print("\nNote: In production, the server witnesses and stores attestations.")
        print("The receipt itself contains the Merkle proof for verification.")
else:
    print("Skipped - no GLACIS_API_KEY")

---
## 2. Control Stack: PII Redaction + Jailbreak Detection

The control stack with:
- **PII/PHI Redaction**: HIPAA Safe Harbor compliant redaction
- **Jailbreak Detection**: Meta Llama Prompt Guard 2

Controls are configured in `glacis.yaml`.

In [ ]:
if GLACIS_API_KEY and OPENAI_API_KEY:
    from glacis.integrations.openai import attested_openai, get_last_receipt

    # Create an attested OpenAI client with controls enabled (via glacis.yaml)
    client = attested_openai(
        glacis_api_key=GLACIS_API_KEY,
        openai_api_key=OPENAI_API_KEY,
        offline=False,  # Use online mode
        debug=True,  # Show control execution
    )
    print("Attested OpenAI client ready (online mode with controls)!")
else:
    missing = []
    if not GLACIS_API_KEY: missing.append("GLACIS_API_KEY")
    if not OPENAI_API_KEY: missing.append("OPENAI_API_KEY")
    print(f"Skipped - missing: {', '.join(missing)}")

In [ ]:
if GLACIS_API_KEY and OPENAI_API_KEY:
    from glacis.integrations.openai import get_evidence

    # Clinical query with PHI (will be redacted)
    system_prompt = """You are a clinical decision support assistant for healthcare providers.
Your role is to help with medication interaction checks and treatment recommendations.
If you see redacted identifiers like [PERSON], [US_SSN], [EMAIL_ADDRESS], or [MRN],
respond helpfully without needing the actual values - focus on the clinical question."""

    clinical_query = """
I need a medication interaction check for my patient.

Patient: Margaret Chen
DOB: 11/22/1956
MRN: MRN-4829173
Contact: margaret.chen@gmail.com, (415) 555-8234

Current Medications:
- Warfarin 5mg daily (target INR 2-3 for afib)
- Metoprolol 50mg twice daily
- Lisinopril 20mg daily
- Atorvastatin 40mg at bedtime

The patient reports worsening knee pain from osteoarthritis. She's asking about
starting over-the-counter ibuprofen (Advil) 400mg three times daily.

What are the key drug interactions I should consider?
"""

    print("=" * 70)
    print("ORIGINAL QUERY (contains PHI)")
    print("=" * 70)
    print(clinical_query)

    print("\n" + "=" * 70)
    print("CALLING OPENAI (with controls)...")
    print("=" * 70 + "\n")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": clinical_query}
        ],
        max_tokens=300,
    )

    # Show what was actually sent to OpenAI (redacted)
    receipt = get_last_receipt()
    evidence = get_evidence(receipt.id)
    if evidence:
        redacted_content = evidence['input']['messages'][1]['content']
        print("=" * 70)
        print("REDACTED QUERY (what OpenAI received)")
        print("=" * 70)
        print(redacted_content)

    print("\n" + "=" * 70)
    print("CLINICAL RESPONSE")
    print("=" * 70)
    print(response.choices[0].message.content)
else:
    print("Skipped - missing API keys")

---
## 3. Control Plane Attestation Report

The attestation receipt includes control plane results showing what controls executed.
In online mode, the receipt also includes Merkle proofs for third-party verification.

In [ ]:
if GLACIS_API_KEY and OPENAI_API_KEY:
    # Get the receipt from the last call
    receipt = get_last_receipt()

    print("=" * 70)
    print("ATTESTATION RECEIPT (ONLINE - WITNESSED)")
    print("=" * 70)
    print(f"  Receipt ID:       {receipt.id}")
    print(f"  Timestamp:        {receipt.timestamp}")
    print(f"  Evidence Hash:    {receipt.evidence_hash[:32]}...")
    print(f"  Leaf Index:       {receipt.leaf_index}")
    print(f"  Tree Size:        {receipt.tree_size}")

    # Show Merkle proof
    if receipt.receipt and receipt.receipt.transparency_proofs:
        proofs = receipt.receipt.transparency_proofs
        print(f"\n  Merkle Proof:")
        print(f"    Root Hash: {proofs.sth_curr.root_hash[:32]}...")
        print(f"    Tree Size: {proofs.sth_curr.tree_size}")

    if receipt.control_plane_results:
        cp = receipt.control_plane_results
        print(f"\n{'='*70}")
        print("CONTROL PLANE RESULTS")
        print(f"{'='*70}")
        print(f"  Policy ID:      {cp.policy.id}")
        print(f"  Policy Version: {cp.policy.version}")
        print(f"  Model:          {cp.policy.model.provider}/{cp.policy.model.model_id}")
        print(f"  Environment:    {cp.policy.scope.environment}")
        print(f"  Tags:           {cp.policy.scope.tags}")

        print(f"\n  Determination:")
        print(f"    Action:     {cp.determination.action}")
        print(f"    Trigger:    {cp.determination.trigger}")
        print(f"    Confidence: {cp.determination.confidence}")

        print(f"\n  Controls Executed: {len(cp.controls)}")
        for ctrl in cp.controls:
            print(f"    - {ctrl.id}")
            print(f"      Type:    {ctrl.type}")
            print(f"      Status:  {ctrl.status}")
            print(f"      Latency: {ctrl.latency_ms}ms")
else:
    print("Skipped - missing API keys")

---
## 4. Third-Party Verification

The badge URL allows **anyone** to verify this attestation without access to your systems.
This is key for regulatory compliance and external audits.

In [ ]:
if GLACIS_API_KEY and OPENAI_API_KEY:
    from glacis import GlacisApiError

    print("=" * 70)
    print("THIRD-PARTY VERIFICATION")
    print("=" * 70)
    print(f"\nReceipt ID: {receipt.id}")
    print("\nAnyone can verify this attestation using the receipt ID.")
    print("No access to your systems required - just the evidence hash.")

    # Demonstrate verification via attestation hash
    print(f"\n" + "-" * 70)
    print("Verifying attestation (as a third-party would)...")
    print("-" * 70)

    try:
        # Pass the full receipt - client extracts evidence_hash for verification
        result = glacis.verify(receipt)

        print(f"\nVerification Result:")
        print(f"  Evidence Hash:    {receipt.evidence_hash[:32]}...")
        print(f"  Overall Valid:    {result.valid}")
        if result.verification:
            print(f"  Signature Valid:  {result.verification.signature_valid}")
            print(f"  Proof Valid:      {result.verification.proof_valid}")

        if result.valid:
            print("\n  This attestation is cryptographically verified!")
            print("  - Signature proves it came from a GLACIS client")
            print("  - Merkle proof proves it's in the transparency log")
            print("  - Timestamp is immutable and verifiable")
    except GlacisApiError as e:
        print(f"\nServer verification: {e}")
        print("\nIn production, third-parties can verify via the API.")
else:
    print("Skipped - missing API keys")

---
## 5. Query Transparency Log

Query past attestations from the transparency log. Useful for:
- Searching historical attestations
- Monitoring AI usage patterns
- Building compliance dashboards

In [ ]:
if GLACIS_API_KEY:
    from glacis import GlacisApiError

    print("=" * 70)
    print("TRANSPARENCY LOG QUERY")
    print("=" * 70)

    try:
        # Query by service_id
        results = glacis.query_log(
            service_id="demo-app",
            limit=10,
        )

        print(f"\nFound {results.count} attestations for service 'demo-app'")
        if results.tree_head:
            print(f"Current tree size: {results.tree_head.size}")
            print(f"Tree root hash:    {results.tree_head.root_hash[:32]}...")

        if results.entries:
            print(f"\nRecent entries:")
            for i, entry in enumerate(results.entries[:5], 1):
                print(f"\n  {i}. {entry.attestation_id}")
                print(f"     Timestamp:  {entry.timestamp}")
                print(f"     Operation:  {entry.operation_type}")
                print(f"     Leaf Index: {entry.leaf_index}")
        else:
            print("\nNo entries found for this service yet.")
    except GlacisApiError as e:
        print(f"\nQuery not available: {e}")
        print("\nIn production, you can query the transparency log for historical attestations.")
else:
    print("Skipped - no GLACIS_API_KEY")

---
## 6. Compliance Export

Export attestations for compliance reporting. Query by time range and export to any format.

In [ ]:
if GLACIS_API_KEY:
    from datetime import datetime, timedelta, UTC
    from glacis import GlacisApiError

    print("=" * 70)
    print("COMPLIANCE EXPORT")
    print("=" * 70)

    try:
        # Query attestations in a time range
        end_time = datetime.now(UTC)
        start_time = end_time - timedelta(days=30)

        results = glacis.query_log(
            service_id="demo-app",
            start=start_time.isoformat(),
            end=end_time.isoformat(),
            limit=100,
        )

        print(f"\nAttestation Summary (Last 30 Days)")
        print(f"-" * 40)
        print(f"  Service:     demo-app")
        print(f"  Time Range:  {start_time.date()} to {end_time.date()}")
        print(f"  Total Count: {results.count}")
        if results.tree_head:
            print(f"  Tree Head:   {results.tree_head.root_hash[:32]}...")

        if results.entries:
            print(f"\n" + "=" * 70)
            print("EXPORT FORMAT (for compliance audit)")
            print("=" * 70)

            for entry in results.entries[:3]:
                print(f"""
  Attestation: {entry.attestation_id}
  Timestamp:   {entry.timestamp or 'N/A'}
  Service:     {entry.service_id or 'N/A'}
  Operation:   {entry.operation_type or 'N/A'}
  Hash:        {entry.payload_hash or 'N/A'}
  Leaf Index:  {entry.leaf_index}
  ---""")

            print(f"\nFull attestation history available via glacis.query_log()")
            print("Export to CSV/JSON for compliance reporting systems")
        else:
            print("\nNo attestations found in this time range.")
            print("Create some attestations first, then query the log.")
    except GlacisApiError as e:
        print(f"\nExport not available: {e}")
        print("\nIn production, query attestation history for compliance reporting.")
else:
    print("Skipped - no GLACIS_API_KEY")

---
## 7. Evidence Retrieval

GLACIS stores full evidence locally for audit trails. This includes:
- The exact input (after redaction)
- The LLM response
- Control plane results

**Storage location**: `~/.glacis/glacis.db` (SQLite)

In [ ]:
if GLACIS_API_KEY and OPENAI_API_KEY:
    from glacis.storage import ReceiptStorage

    storage = ReceiptStorage()
    evidence = storage.get_evidence(receipt.id)

    if evidence:
        print("=" * 70)
        print("FULL EVIDENCE (Stored Locally)")
        print("=" * 70)
        print(f"\nAttestation ID:   {evidence['attestation_id']}")
        print(f"Attestation Hash: {evidence['attestation_hash'][:32]}...")
        print(f"Mode:             {evidence['mode']}")
        print(f"Service ID:       {evidence['service_id']}")
        print(f"Timestamp:        {evidence['timestamp']}")

        print(f"\n--- INPUT (what OpenAI received - REDACTED) ---")
        input_data = evidence['input']
        print(f"Model: {input_data.get('model')}")
        for msg in input_data.get('messages', []):
            role = msg.get('role', 'unknown')
            content = msg.get('content', '')
            preview = content[:300] + "..." if len(content) > 300 else content
            print(f"\n[{role}]:")
            print(preview)

        print(f"\n--- OUTPUT (LLM response) ---")
        output_data = evidence['output']
        if output_data.get('choices'):
            content = output_data['choices'][0].get('message', {}).get('content', '')
            preview = content[:400] + "..." if len(content) > 400 else content
            print(preview)

        if output_data.get('usage'):
            usage = output_data['usage']
            print(f"\nTokens: {usage.get('prompt_tokens')} prompt + {usage.get('completion_tokens')} completion")

        print(f"\n" + "=" * 70)
        print("Evidence stored at: ~/.glacis/glacis.db")
        print("Retrieve anytime: ReceiptStorage().get_evidence(receipt_id)")
    else:
        print(f"No evidence found for: {receipt.id}")
else:
    print("Skipped - missing API keys")

In [ ]:
# # Cleanup
# if GLACIS_API_KEY:
#     glacis.close()
# print("Done!")

---
## Summary: Online Mode Benefits

With a GLACIS API key, you get:

| Feature | What It Means |
|---------|---------------|
| **Server-Witnessed Receipts** | Attestations are recorded in the GLACIS transparency log |
| **Merkle Proofs** | Cryptographic proof your attestation exists in the log |
| **Badge URL** | Anyone can verify without access to your systems |
| **Query Transparency Log** | Search historical attestations by service, time range |
| **Compliance Export** | Export attestation history for auditors |
| **Tamper Evidence** | Any modification to the log changes the Merkle root |

**Get your API key at [https://glacis.io](https://glacis.io)**